# Theoretical Questions

* This is the theoretical part of the final project. It includes theoretical questions from various topics covered in the course.
* There are 7 questions among which you need to choose 6, according to the following key:
    + Question 1 is **mandatory**.
    + Choose **one question** from questions 2-3.
    + Question 4 is **mandatory**.
    + Questions 5-6 are **mandatory**.
    + Question 7 is **mandatory**.
* Question 1 is worth 15 points, whereas the other questions worth 7 points.
* All in all, the maximal grade for this parts is 15+7*5=50 points.
* **You should answer the questions on your own. We will check for plagiarism.**
* If you need to add external images (such as graphs) to this notebook, please put them inside the 'imgs' folder. DO NOT put a reference to an external link.
* Good luck!

## Part 1: General understanding of the course material

### Question 1

1.  Relate the number of parameters in a neural network to the over-fitting phenomenon (*).
    Relate this to the design of convolutional neural networks, and explain why CNNs are a plausible choice for an hypothesis class for visual classification tasks.

    (*) In the context of classical under-fitting/over-fitting in machine learning models.

### 1.1 Answer:
A large number of parameters can lead to overfitting, where the model learns noise from the training data and generalizes poorly, because the model can memorize the training data.
Convolutional Neural Networks (CNNs) use spatial hierarchies and shared weights to reduce the number of free parameters. This inherently regularizes the model, making it less prone to overfitting and making CNNs a natural choice for image and visual classification tasks.


2. Consider the linear classifier model with hand-crafted features: $$f_{w,b}(x) = w^T \psi(x) + b$$

  where $x \in \mathbb{R}^2$, $\psi$ is a non-learnable feature extractor and assume that the classification is done by $sign(f_{w,b}(x))$. Let $\psi$ be the following feature extractor $\psi(x)=x^TQx$ where $Q \in \mathbb{R}^{2 \times 2}$ is a non-learnable positive definite matrix. Describe a distribution of the data which the model is able to approximate, but the simple linear model fails to approximate (hint: first, try to describe the decision boundary of the above classifier).



    

### 1.2 Answer:
The given model is a linear classifier acting on transformed features. Specifically, the features are transformed by the quadratic function $\psi(x) = x^T Q x$, where $Q$ is a positive definite matrix.

The decision boundary of the classifier is the set of points where $f_{w,b}(x) = 0$, or equivalently,
$w^T \psi(x) + b = 0.$

Substituting the expression for $\psi(x)$ into the above equation, we have:
$w^T (x^T Q x) + b = 0.$



Since $Q$ is a positive definite matrix, we can express $Q$ as $Q = A^T A$ for some invertible matrix $A$. Substituting this into the above expression, we have:
$w^T x^T A^T A x + b = 0.$

Or equivalently:
$w^T (A x)^T (A x) + b = 0.$

This equation describes a quadratic decision boundary in $x$, which means that the decision boundary is not a hyperplane but a more general quadratic surface.


Consider the distribution of data where the points are arranged in two concentric circles, with one class inside the inner circle and the other class outside it. If the matrix $Q$ is chosen such that the decision boundary corresponds to these concentric circles, then the given model can approximate this distribution. Another possibility is that the data is arranged in the form of a Parabola so that one class is inside and one is outside.

The simple linear model, however, cannot represent this decision boundary, as it is constrained to linear decision boundaries, and will thus fail to approximate this distribution of data.



3. Assume that we would like to train a Neural Network for classifying images into $C$ classes. Assume that the architecture can be stored in the memory as a computational graph with $N$ nodes where the output is the logits (namely, before applying softmax) for the current batch ($f_w: B \times Ch \times H \times W \rightarrow B \times C$). Assume that the computational graph operates on *tensor* values.
    * Implement the CE loss assuming that the labels $y$ are hard labels given in a LongTensor of shape $B \times 1$. **Use Torch's log_softmax and gather functions** and implement with less as possible operations.

In [ ]:
from torch.nn.functional import log_softmax
from torch import gather
# Input:  model, x, y.
# Output: the loss on the current batch.

# Assume this returns the logits
logits = model(x)

# Compute log probabilities
log_probs = log_softmax(logits, dim=1)

# Extract the log probabilities of the true labels
loss_values = gather(log_probs, 1, y)

# Average the negative log probabilities to get the loss
loss = -loss_values.mean()

# loss now contains the cross-entropy loss of the batch


* Using the model's function as a black box, draw the computational graph (treating both log_softmax and gather as an atomic operations). How many nodes are there in the computational graph?

### 1.3.2 answer:
It's hard for me to draw, so I described in words:

1. Node for `model(x)`: This represents the forward pass of the model that outputs logits. Input edge is `x`, and the output edge is `logits`.

2. Node for `log_softmax`: Takes `logits` as an input edge and outputs `log_probs`.

3. Node for `gather`: Takes `log_probs` and `y` as input edges, and outputs `loss_values`.

4. Node for `mean`: Takes `loss_values` as an input edge and outputs the `loss`.

there are 4 nodes in the computational graph:

- model(x)
- log_softmax
- gather
- mean



* Now, instead of using hard labels, assume that the labels are representing some probability distribution over the $C$ classes. How would the gradient computation be affected? analyze the growth in the computational graph, memory and computation.

 ### 1.3.3 answer:
 Instead of using `gather` to pick out the log probabilities of the true classes according to hard labels, we would compute the weighted sum of all log probabilities, weighted by the label probabilities.

The cross-entropy loss for soft labels can be defined as:

$\text{Loss} = -\sum_{i=1}^{B}\sum_{c=1}^{C} p_{i,c} \log(q_{i,c})$

Where $ p_{i,c} $ is the true distribution (of the soft labels) and $ q_{i,c} $ is the estimated distribution (obtained from softmax on the logits).

Here's a PyTorch snippet to compute the cross-entropy loss using soft labels:

```python
import torch
from torch.nn.functional import log_softmax

logits = model(x)
log_probs = log_softmax(logits, dim=1)

# Cross-Entropy Loss with soft labels
loss = -(soft_labels * log_probs).sum(dim=1).mean()
```

### Computational Graph

1. Node for `model(x)`: Outputs `logits`.
2. Node for `log_softmax`: Takes `logits` and outputs `log_probs`.
3. Node for `*`: Element-wise multiplication between `soft_labels` and `log_probs`, outputs a temporary tensor.
4. Node for `sum(dim=1)`: Sums along the class dimension, outputs a tensor of shape `B`.
5. Node for `mean`: Takes the mean of the tensor of shape `B`, outputs the `loss`.

There would be 5 nodes instead of 4 nodes in the computational graph.

### Memory  

The memory footprint would increase slightly:

1. `soft_labels`: An additional `B x C` tensor to store the soft labels.
2. Temporary tensor for storing the element-wise product: `B x C`.

### Computational  

1. Element-wise multiplication between `soft_labels` and `log_probs`: $ O(B \times C) $.
2. Sum along the class dimension: $ O(B \times C) $.
3. Mean across the batch: $ O(B) $.

The computational complexity is slightly higher due to the extra element-wise multiplication and sum operations, both of which are $ O(B \times C) $.



* Apply the same analysis in the case that we would like to double the batch size. How should we change the learning rate of the optimizer?

### 1.3.4 answer:
### Computational Graph
The nodes in the computational graph remain the same; only the dimensions of the tensors flowing through the graph change. So, we still have the same 4 or 5 nodes (depending on whether we are using hard or soft labels).

### Memory  
Doubling the batch size will roughly double the memory requirements for each tensor involved in the batch operation. we will need twice as much memory to store `logits`, `log_probs`, and `loss_values` (or the temporary tensor in case of soft labels). This can be particularly concerning if our model and/or dataset are already large, as we may run into memory limitations.

### Computational  
1. Computational cost for forward and backward passes through the model will roughly double.
2. Element-wise operations, such as multiplication and summation, will also take roughly twice as long.


### Learning Rate
When we increase the batch size, each gradient estimate is likely to be more accurate (closer to the true gradient) because it is computed based on more data. As a result, the optimizer can afford to take a larger step during each iteration, suggesting that we may increase the learning rate (the relationship between batch size and learning rate is not necessarily linear, and the optimal adjustment can be problem-dependent).

## Part 2: Optimization & Automatic Differentiation

### Question 2: resolving gradient conflicts in multi-task learning

Assume that you want to train a model to perform two tasks: task 1 and task 2.
For each such task $i$ you have an already implemented function *loss\_i = forward_and_compute_loss_i(model,inputs)* such that given the model and the inputs it computes the loss w.r.t task $i$ (assume that the computational graph is properly constructed). We would like to train our model using SGD to succeed in both tasks as follows: in each training iteration (batch) -
* Let $g_i$ be the gradient w.r.t the $i$-th task.
* If $g_1 \cdot g_2 < 0$:
    + Pick a task $i$ at random.
    + Apply GD w.r.t only that task.
* Otherwise:
    + Apply GD w.r.t both tasks (namely $\mathcal{L}_1 + \mathcal{L}_2$).

Note that in the above formulation the gradient is a thought of as a concatination of all the gradient w.r.t all the models parameters, and $g_1 \cdot g_2$ stands for a dot product.

What parts should be modified to implement the above? Is it the optimizer, the training loop or both? Implement the above algorithm in a code cell/s below

### 2 answer:
The described algorithm involves a custom decision-making process in each batch iteration to decide whether to update the model's weights based on one task or both tasks. Given this, the core logic for this multi-task optimization cannot be encapsulated by a standard optimizer alone. Therefore, we would need to modify the training loop to implement this algorithm. The optimizer can still be a standard one (like SGD) but its application is conditional based on the dot product of the gradients $g_1 \cdot g_2$.

the algorithm:


In [ ]:
import torch
import random

# Initialize model, inputs and optimizer
model = Model()
inputs = Inputs()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Training loop (Assume 10 times)
for epoch in range(10):

    # Compute losses for both tasks
    loss1 = forward_and_compute_loss_1(model, inputs)
    loss2 = forward_and_compute_loss_2(model, inputs)

    # Zero the gradients before computing the task's gradients and compute gradients for both tasks
    optimizer.zero_grad()
    loss1.backward(retain_graph=True)
    grad1 = torch.cat([p.grad.view(-1) for p in model.parameters()])

    optimizer.zero_grad()
    loss2.backward(retain_graph=True)
    grad2 = torch.cat([p.grad.view(-1) for p in model.parameters()])

    # Zero the gradients again before final update
    optimizer.zero_grad()

    # Compute dot product
    dot_product = torch.dot(grad1, grad2)

    if dot_product < 0:
        # Pick a task at random and apply gradient descent only w.r.t that task
        task_choice = random.choice([1, 2])
        if task_choice == 1:
            loss1.backward()
        else:
            loss2.backward()
    else:
        # Apply GD w.r.t both tasks (loss1 + loss2)
        total_loss = loss1 + loss2
        total_loss.backward()

    # Update the model parameters (new parameter = old parameter − learning rate)
    optimizer.step()


### Question 3: manual automatic differentiation

Consider the following two-input two-output function:
$$ f(x,y) = (x^2\sin(xy+\frac{\pi}{2}), x^2\ln(1+xy)) $$
* Draw a computational graph for the above function. Assume that the unary atomic units are squaring, taking square root, $\exp,\ln$, basic trigonometric functions and the binary atomic units are addition and multiplication. You would have to use constant nodes.
* Calculate manually the forward pass.
* Calculate manually the derivative of all outputs w.r.t all inputs using a forward mode AD.
* Calculate manually the derivative of all outputs w.r.t all inputs using a backward mode AD.

## Part 3: Sequential Models

### Question 4: RNNs vs Transformers in the real life

In each one of the following scenarios decide whether to use RNN based model or a transformer based model. Justify your choice.
1. You are running a start-up in the area of automatic summarization of academic papers. The inference of the model is done on the server side, and it is very important for it to be fast.
2. You need to design a mobile application that gathers small amount of data from few apps in every second and then uses a NN to possibly generate an alert given the information in the current second and the information from the past minute.
3. You have a prediction task over fixed length sequences on which you know the following properties:
    + In each sequence there are only few tokens that the model should attend to.
    + Most of the information needed for generating a reliable prediction is located at the beginning of the sequence.
    + There is no restriction on the computational resources.

### 4 answer:
***1: Quick automatic summarization of academic papers***

**Choice**: Transformer-based model

**Justification**: Transformers have been demonstrated to perform exceptionally well in natural language processing tasks like text summarization, often outperforming RNNs in terms of accuracy and quality. Additionally, Transformer models are highly parallelizable, which can make inference faster when computational resources are optimized. Speed is of essence here, as stated in our requirement, and Transformer-based architectures have shown the ability to produce fast and high-quality summaries.


***2: Mobile application for real-time alerts***

**Choice**: RNN-based model

**Justification**: RNNs are generally more lightweight compared to Transformers, making them a good choice for mobile applications where computational resources are limited. In the scenario described, we are dealing with small amounts of sequential data to possibly generate real-time alerts. RNNs are well-suited for sequence-based data and can be highly optimized to make quick predictions. Given the low-latency requirement and the real-time nature of the task, a well-optimized RNN should suffice for generating reliable and timely alerts.


***3: Prediction over fixed length sequences***

**Choice**: Transformer-based model

**Justification**: The conditions of this problem are quite favorable for Transformer models:

- *Few tokens to attend to*: Transformers are good at focusing on specific parts of the input due to their self-attention mechanism. If only a few tokens are relevant, the attention mechanism can be particularly useful.
  
- *Information at the beginning*: The Transformer doesn't have to "remember" the beginning throughout the sequence like an RNN does. It can simply attend to the beginning tokens when it needs to.

- *No restriction on computational resources*: Transformers tend to require more computational resources than RNNs, but you have stated that this is not an issue in this case.

Given these conditions, a Transformer-based model is more likely to produce more accurate and reliable predictions for this particular scenario.
  

## Part 4: Generative modeling

### Question 5: VAEs and GANS

Suggest a method for combining VAEs and GANs. Focus on the different components of the model and how to train them jointly (the objectives). Which drawbacks of these models the combined model may overcome? which not?

### 5. answer:

#### Components of the Model
- Encoder (E): Part of VAE, responsible for encoding an input image $x$ into a latent vector $z$ with associated mean $ \mu $ and standard deviation $ \sigma $.
- Decoder (D): Part of VAE, responsible for reconstructing the input image from the latent vector $z$.
- Generator (G): Part of GAN, responsible for generating fake samples from a noise vector.
- Discriminator (Dis): Part of GAN, responsible for distinguishing real samples from fake samples.

#### Joint Training Objectives

- VAE Loss: The VAE part includes a reconstruction loss (e.g., mean squared error between the input and reconstructed output) and a KL divergence term that encourages the latent variables to follow a specific distribution (usually standard normal).

- GAN Loss: The GAN part includes the typical adversarial loss. The discriminator tries to differentiate between real and generated samples, while the generator tries to fool the discriminator.

- **Combined Training Objective ($\mathcal{L}$)**: The final objective is a weighted combination of the VAE loss and GAN loss. The model can be trained end-to-end with gradient descent.


  

### Training Procedure
- Forward pass: Input $ x $ through the Encoder to obtain $ \mu $ and $ \sigma $ and sample $ z $ from the distribution $ q(z|x) $.
- Decoder pass: Reconstruct $ \hat{x} $ from $ z $.
- GAN pass: Use the same $ z $ or another noise $ z' $ to generate $ G(z) $ or $ G(z') $ and use Discriminator $ D $ to distinguish between real and fake samples.
- Update Encoder, Decoder, Generator, and Discriminator by backpropagation based on $ \mathcal{L} $.

### Benefits

- Improved Generalization: GANs can offer the VAE model the capability to produce sharper, more realistic samples.
- Regularization: The VAE’s regularization term can help the GAN in learning a more interpretable and smooth latent space.
- Stability: The reconstruction loss from the VAE can make training more stable, mitigating mode collapse that GANs often suffer from.

### Drawbacks

- Training Complexity: Training would be more challenging due to conflicting objectives and the need to balance them appropriately.
- Computationally Intensive: More network components require more computational resources.

### Drawbacks Not Overcome

- Disentanglement: Neither GANs nor VAEs are explicitly designed to learn disentangled representations.
- Evaluation Metrics: Since we combine both models, developing or choosing an appropriate metric for evaluation would be hard.




### Question 6: Diffusion Models

Show that $q(x_{t-1}|x_t,x_0)$ is tractable and is given by $\mathcal{N}(x_{t-1};\tilde{\mu}(x_t,x_0),\tilde{\beta_t}I)$ where the terms for $\tilde{\mu}(x_t,x_0)$ and $\tilde{\beta_t}$ are given in the last tutorial. Do so by explicitly computing the PDF.

### 6 answer:

We'll start with the Bayesian formula for $ q(x_{t-1}|x_t, x_0) $:

$
q(x_{t-1}|x_t, x_0) = \frac{q(x_t|x_{t-1}) q(x_{t-1}|x_0)}{q(x_t|x_0)}
$

Each term is a Gaussian density, and the Gaussian density for a random variable $ y $ with mean $ \mu $ and variance $ \sigma^2 $ is given by:

$
\mathcal{N}(y; \mu, \sigma^2) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp \left( -\frac{(y - \mu)^2}{2\sigma^2} \right)
$

For the individual terms, we have:

1. $ q(x_t|x_{t-1}) =  \mathcal{N}(x_t; \sqrt{1-\beta_t} x_{t-1}, \beta_t \mathbf{I})  = \frac{1}{\sqrt{2\pi\beta_t}} \exp \left( -\frac{(x_t - \sqrt{1-\beta_t}x_{t-1})^2}{2\beta_t} \right) $
2. $ q(x_{t-1}|x_0) =  \mathcal{N}(x_{t-1};\sqrt{\bar{\alpha}_{t-1}}x_0, (1-\bar{\alpha}_{t-1})\mathbf{I}) = \frac{1}{\sqrt{2\pi(1-\bar{\alpha}_{t-1})}} \exp \left( -\frac{(x_{t-1} - \sqrt{\bar{\alpha}_{t-1}}x_0)^2}{2(1-\bar{\alpha}_{t-1})} \right) $
3. $ q(x_t|x_0) = \mathcal{N}(x_t;\sqrt{\bar{\alpha_t}}x_0, (1-\bar{\alpha_t})\mathbf{I}) = \frac{1}{\sqrt{2\pi(1-\bar{\alpha}_t)}} \exp \left( -\frac{(x_t - \sqrt{\bar{\alpha}_t}x_0)^2}{2(1-\bar{\alpha}_t)} \right) $

Calculate the $ q(x_{t-1}|x_t, x_0) $:

$
q(x_{t-1}|x_t, x_0) = \frac{q(x_t|x_{t-1}) q(x_{t-1}|x_0)}{q(x_t|x_0)} = \frac{1}{\sqrt{2\pi\beta_t}}\frac{\sqrt{2\pi(1-\bar{\alpha}_t)}}{\sqrt{2\pi(1-\bar{\alpha}_{t-1})}}\exp\left( -\frac{(x_t - \sqrt{1-\beta_t} x_{t-1})^2}{2\beta_t} \right) \exp \left( -\frac{(x_{t-1} - \sqrt{\bar{\alpha}_{t-1}} x_0)^2}{2(1-\bar{\alpha}_{t-1})} \right)\exp\left( \frac{(x_t - \sqrt{\bar{\alpha}_t}x_0)^2}{2(1-\bar{\alpha}_t)} \right) = \frac{1}{\sqrt{2\pi\tilde{\beta_t}}}\exp \left( -\frac{1}{2} \left( \frac{(x_t - \sqrt{1-\beta_t} x_{t-1})^2}{\beta_t} + \frac{(x_{t-1} - \sqrt{\bar{\alpha}_{t-1}} x_0)^2}{1-\bar{\alpha}_{t-1}} - \frac{(x_t - \sqrt{\bar{\alpha}_t}x_0)^2}{1-\bar{\alpha}_t} \right) \right) =
 \frac{1}{\sqrt{2\pi\tilde{\beta_t}}}\exp \left( -\frac{1}{2} \left( \frac{(x_{t-1} - \frac{\sqrt{\bar{\alpha}_{t-1}}\beta_t}{1-\bar{\alpha}_t}x_0 + \frac{\sqrt{\alpha_t}(1-\bar{\alpha}_{t-1})}{1-\bar{\alpha}_t}x_t)^2}{\frac{1-\bar{\alpha}_{t-1}}{1-\bar{\alpha}_t} \beta_t} \right) \right) = \frac{1}{\sqrt{2 \pi \tilde{\beta_t}}} \exp \left(-\frac{(x_{t-1} - \tilde{\mu}(x_t, x_0))^2}{2\tilde{\beta_t}}\right) =  \mathcal{N}(x_{t-1};\tilde{\mu}(x_t,x_0),\tilde{\beta_t}I).
$

###tractability
The tractability of $ q(x_{t-1}|x_t, x_0) $ essentially comes down to being able to compute it efficiently and exactly as we have seen. this tractability is mainly due to two factors:

- Analytic Form: Both the forward and reverse processes are Gaussian processes.  because each step is Gaussian, the conditional distribution $ q(x_{t-1}|x_t, x_0) $ can be computed exactly, preserving the Gaussian form.

- Computational Efficiency: The mean and variance for the Gaussian distribution $ q(x_{t-1}|x_t, x_0) $ can be computed directly using the parameters from the forward and reverse processes by simple algebra operations.



## Part 5: Training Methods

### Question 7: Batch Normalization and Dropout

For both BatchNorm and Dropout analyze the following:
1. How to use them during the training phase (both in forward pass and backward pass)?
2. How differently they behave in the inference phase? How to distinguish these operation modes in code?
3. Assume you would like to perform multi-GPU training (*) to train your model. What should be done in order for BatchNorm and dropout to work properly? assume that each process holds its own copy of the model and that the processes can share information with each other.

(*): In a multi-GPU training each GPU is associated with its own process that holds an independent copy of the model. In each training iteration a (large) batch is split among these processes (GPUs) which compute the gradients of the loss w.r.t the relevant split of the data. Afterwards, the gradients from each process are then shared and averaged so that the GD would take into account the correct gradient and to assure synchornization of the model copies. Note that the proccesses are blocked between training iterations.

### 7 answer:


#### 1. Training Phase

**Batch Normalization**
- **Forward Pass**: During the forward pass, the mean and variance of the mini-batch are calculated. These are then used to normalize the input features. Finally, the normalized features are scaled and shifted using learnable parameters (gamma and beta).

    \begin{equation}
    \text{normalized_input} = \frac{x - \text{mean}}{\sqrt{\text{variance} + \epsilon}}
    \end{equation}

    \begin{equation}
    \text{Batch_norm}(x) = \gamma \times \text{normalized_input} + \beta
    \end{equation}


- **Backward Pass**: The gradients with respect to gamma, beta, mean, and variance are calculated, and then the gradients with respect to the input are calculated. This information is used to update gamma and beta during the optimization process.

**Dropout**
- **Forward Pass**: Randomly set a fraction of input units to 0 during training, to prevent overfitting.

    \begin{equation}
    \text{Dropout}(x) = x \odot \text{mask}
    \end{equation}
  
  where $\odot$ is element-wise multiplication and 'mask' is a tensor of the same shape as 'x' containing random zeros and ones.

- **Backward Pass**: The same mask used in the forward pass is used to propagate the gradients during the backward pass.

#### 2. Inference Phase
**Batch Normalization**
- BatchNorm behaves differently during inference. The mini-batch statistics are not used; instead, running estimates of the mean and variance (usually calculated as an exponential moving average during training) are used.

**Dropout**
- Dropout is usually turned off, i.e., every neuron is used (no dropout).


 *Code*:
  - the mode can be toggled using `model.train()` (sets the model to training mode) and `model.eval()` (sets the model to evaluation) methods.

  - When the model is in training mode (model.train()), Dropout layers will randomly zero out some elements, and BatchNorm layers will normalize based on mini-batch statistics and update running statistics. When the model is in evaluation mode (model.eval()), Dropout layers will not change any elements, and BatchNorm layers will normalize based on running statistics collected during training.



#### 3. Multi-GPU Training
**Batch Normalization**
- When using multi-GPU training, each GPU computes its own mini-batch mean and variance. These statistics could differ significantly between GPUs, leading to model desynchronization.
- The solution is to synchronize the mean and variance computations across all GPUs, averaging these statistics before performing the BatchNorm operation.

**Dropout**
- Dropout generally works well with multi-GPU settings without the need for special adjustments, as each GPU would be doing its own dropout independently.
- Need to ensure is that all the GPUs agree on turning dropout on or off, which is typically controlled by the master process and the 'train()' or 'eval()' method calls.



